In [2]:
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import os

from PIL import Image
from torchvision.transforms import Resize, Compose, ToTensor, Normalize
import numpy as np
import skimage
import matplotlib.pyplot as plt

import time

import random


In [15]:
# print(train_models["RESV_109.npy"][0].net[0].linear.weight.flatten().mean())
# print(train_models["RESV_109.npy"][0].net[0].linear.weight.flatten().var())

# print(train_models["RESV_109.npy"][0].net[1].linear.weight.flatten().mean())
# print(train_models["RESV_109.npy"][0].net[1].linear.weight.flatten().var())

# print(train_models["RESV_109.npy"][0].net[2].linear.weight.flatten().mean())
# print(train_models["RESV_109.npy"][0].net[2].linear.weight.flatten().var())

# print()
# print()

# print(pcmra_siren.net[0].linear.weight.flatten().mean())
# print(pcmra_siren.net[0].linear.weight.flatten().var())
# print(pcmra_siren.net[1].linear.weight.flatten().mean())
# print(pcmra_siren.net[1].linear.weight.flatten().var())
# print(pcmra_siren.net[2].weight.flatten().mean())
# print(pcmra_siren.net[2].weight.flatten().var())

# print()
# print()

# print(mask_siren.net[0].linear.weight.flatten().mean())
# print(mask_siren.net[0].linear.weight.flatten().var())
# print(mask_siren.net[1].linear.weight.flatten().mean())
# print(mask_siren.net[1].linear.weight.flatten().var())
# print(mask_siren.net[2].weight.flatten().mean()
# print(mask_siren.net[2].weight.flatten().var())

[3, 2, 8, 15, 19, 10, 9, 18, 12, 4, 13, 1, 5, 6, 14, 17, 7, 0, 11, 16]


In [2]:
def print_params():
    print("Model1")
    for param in m1.parameters():
        print(param)
        
    print()
    
    print("Model2")
    for param in m2.parameters():
        print(param)
        
    print()
    
    print("Complete Model")
    for param in cm.parameters():
        print(param)

In [3]:
class Model1(torch.nn.Module):
    def __init__(self):
        super(Model1, self).__init__()
        
        self.net = nn.Sequential(nn.Linear(2,3), 
                                 nn.ReLU())
        
    def forward(self, x):  
        return self.net(x)
    

class Model2(torch.nn.Module):
    def __init__(self):
        super(Model2, self).__init__()
        
        self.net = nn.Sequential(nn.Linear(3,1), 
                                 nn.ReLU())
        
    def forward(self, x):  
        return self.net(x)
    
    

    
class CompleteModel(torch.nn.Module):
    def __init__(self):
        super(CompleteModel, self).__init__()
        
        self.net = nn.Sequential(nn.Linear(2,3), 
                                 nn.ReLU(),
                                 nn.Linear(3,1), 
                                 nn.ReLU())
        
        
    def forward(self, x):  
        return self.net(x)    

In [4]:
data = torch.tensor([[0, 0], [1, 1]], dtype=torch.float).cuda()

labels = torch.tensor([[0], [3]], dtype=torch.float).cuda()


In [5]:

random.seed(1)

m1 = Model1().cuda()
m2 = Model2().cuda()
cm = CompleteModel().cuda()

optim1 = torch.optim.Adam(lr=1, params=m1.parameters())
optim2 = torch.optim.Adam(lr=1, params=m2.parameters())
optim = torch.optim.Adam(cm.parameters(), lr=1)

criterion = torch.nn.MSELoss()

print_params()

Model1
Parameter containing:
tensor([[ 0.0239, -0.1268],
        [-0.5727,  0.3217],
        [-0.2448,  0.6367]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.1986, -0.0778, -0.2102], device='cuda:0', requires_grad=True)

Model2
Parameter containing:
tensor([[ 0.3919, -0.0908,  0.5441]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([-0.3409], device='cuda:0', requires_grad=True)

Complete Model
Parameter containing:
tensor([[ 0.5229, -0.3609],
        [-0.3116, -0.3254],
        [ 0.0399, -0.5116]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.3407, -0.6120, -0.6978], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[0.5647, 0.4746, 0.3597]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.1372], device='cuda:0', requires_grad=True)


In [7]:
weight1 = nn.Parameter(torch.Tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]])).cuda()
bias1 = nn.Parameter(torch.Tensor([ 0.2928, -0.0928,  0.5697])).cuda()

weight2 = nn.Parameter(torch.Tensor([[ 0.5473, -0.1857, -0.1335]])).cuda()
bias2 = nn.Parameter(torch.Tensor([0.3104])).cuda()


with torch.no_grad():
    m1.net[0].weight.data = weight1.clone()
    m1.net[0].bias.data = bias1.clone()

    cm.net[0].weight.data = weight1.clone()
    cm.net[0].bias.data = bias1.clone()

    m2.net[0].weight.data = weight2.clone()
    m2.net[0].bias.data = bias2.clone()

    cm.net[2].weight.data = weight2.clone()
    cm.net[2].bias.data = bias2.clone()

m1.train()
m2.train()
cm.train()

CompleteModel(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=True)
    (1): ReLU()
    (2): Linear(in_features=3, out_features=1, bias=True)
    (3): ReLU()
  )
)

In [8]:
print_params()

Model1
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)

Model2
Parameter containing:
tensor([[ 0.5473, -0.1857, -0.1335]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3104], device='cuda:0', requires_grad=True)

Complete Model
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.5473, -0.1857, -0.1335]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3104], device='cuda:0', requires_grad=True)


In [9]:
out = m1(data)
out = m2(out)

loss = criterion(out, labels)

optim1.zero_grad()
optim2.zero_grad()

loss.backward()
print(loss)

tensor(3.5838, device='cuda:0', grad_fn=<MseLossBackward>)


In [10]:
print_params()

Model1
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)

Model2
Parameter containing:
tensor([[ 0.5473, -0.1857, -0.1335]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3104], device='cuda:0', requires_grad=True)

Complete Model
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.5473, -0.1857, -0.1335]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3104], device='cuda:0', requires_grad=True)


In [11]:
optim2.step()

In [12]:
print_params()

Model1
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)

Model2
Parameter containing:
tensor([[ 1.5473, -0.1857,  0.8665]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1.3104], device='cuda:0', requires_grad=True)

Complete Model
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.5473, -0.1857, -0.1335]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3104], device='cuda:0', requires_grad=True)


In [13]:
optim1.step()

In [14]:
print_params()

Model1
Parameter containing:
tensor([[ 0.3634,  1.4821],
        [-0.5287, -0.3526],
        [-0.8491, -1.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 1.2928, -0.0928, -0.4303], device='cuda:0', requires_grad=True)

Model2
Parameter containing:
tensor([[ 1.5473, -0.1857,  0.8665]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1.3104], device='cuda:0', requires_grad=True)

Complete Model
Parameter containing:
tensor([[-0.6366,  0.4821],
        [-0.5287, -0.3526],
        [ 0.1509, -0.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 0.2928, -0.0928,  0.5697], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 0.5473, -0.1857, -0.1335]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3104], device='cuda:0', requires_grad=True)


In [15]:
out = cm(data)

loss = criterion(out, labels)

optim.zero_grad()

loss.backward()
optim.step()
print(loss)

tensor(3.5838, device='cuda:0', grad_fn=<MseLossBackward>)


In [16]:
print_params()

Model1
Parameter containing:
tensor([[ 0.3634,  1.4821],
        [-0.5287, -0.3526],
        [-0.8491, -1.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 1.2928, -0.0928, -0.4303], device='cuda:0', requires_grad=True)

Model2
Parameter containing:
tensor([[ 1.5473, -0.1857,  0.8665]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1.3104], device='cuda:0', requires_grad=True)

Complete Model
Parameter containing:
tensor([[ 0.3634,  1.4821],
        [-0.5287, -0.3526],
        [-0.8491, -1.4653]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([ 1.2928, -0.0928, -0.4303], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([[ 1.5473, -0.1857,  0.8665]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([1.3104], device='cuda:0', requires_grad=True)


In [3]:
a = torch.eye(4)

In [4]:
b = torch.Tensor([0, 1, 2, 3, 4, 5])

In [5]:
c = a.to(b)

In [6]:
print(c)

tensor([[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]])
